# Word Mover's Distance 
## по статье Matthew J. Kusner'а "From Word Embeddings to Document Distances" [1]

![img](https://raw.githubusercontent.com/mkusner/wmd/master/fig1.png)


Формулировка задачи определения сходства между двумя предложениями как задачи транспортной задачи:

1. Пусть $X \in \mathbb{R}^{d \times n}$ – матрица эмбеддингов,  $d$ – размерность эмбеддинга, $n$ - количество слов;
2. Вектор-документ в векторной модели: $d \in \mathbb{R}^n$ состоит из $c_i = \texttt{count}(word_i, doc)$
3. Нормированный вектор-документ: $d_i = \frac{c_i}{\sum_i c_i}$
4. Расстояние между словами: $\texttt{cost}(word_i, word_j) = ||x_i - x_j||_2$

Дано два документа, $d, d'$. Пусть  $T \in \mathbb{R}^{n \times n}$, $T_{ij} \ge 0$ – матрица потока показывает расстояния от каждого слова $d$ до $d'$.

Транспортная задача:

$\min_{T \ge 0} \sum_{i,j}^n T_{ij}\texttt{cost}(word_i, word_j) $

при условии:

$\sum_{j} T_{ij} = d_i$

$\sum_{i} T_{ij} = d'_j$.

Задача решается средствами линейного программирования.

## Примеры 

In [1]:
from time import time
start_nb = time()

Используем fasttext эмбеддинги Facebook:

In [2]:
start = time()
import os

from gensim.models import *

model = KeyedVectors.load_word2vec_format('/NLP/embeddings/wiki.ru.vec', binary=False)

print('Cell took %.2f seconds to run.' % (time() - start))

/Library/Frameworks/Python.framework/Versions/3.6/lib/python3.6/site-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
Using TensorFlow backend.


Cell took 503.74 seconds to run.


Вычисляем попарное сходство между тремя тестовыми предложениями:

In [3]:
s1 = 'Ученые обнаружили ископаемую ящерицу с парой теменных глаз'
s2 = 'У палеогеновой ящерицы нашли вторую пару глаз'
s3 = 'Apple через два года откажется от процессоров Intel'

distance = model.wmdistance(s1, s2)
print ('distance between s1 and s2 = %.4f' % distance)

distance = model.wmdistance(s1, s3)
print ('distance between s1 and s3 = %.4f' % distance)

distance = model.wmdistance(s2, s3)
print ('distance between s2 and s3 = %.4f' % distance)

distance between s1 and s2 = 1.3363
distance between s1 and s3 = 2.5128
distance between s2 and s3 = 2.5454


In [ ]:
Вычисляем попарное сходство между тремя тестовыми предложениями с использованием нормированных эмбеддингов:

In [4]:
model.init_sims(replace=True)  

distance = model.wmdistance(s1, s2)
print ('distance between s1 and s2 = %.4f' % distance)

distance = model.wmdistance(s1, s3)
print ('distance between s1 and s3 = %.4f' % distance)

distance = model.wmdistance(s2, s3)
print ('distance between s2 and s3 = %.4f' % distance)

distance between s1 and s2 = 0.3252
distance between s1 and s3 = 0.5703
distance between s2 and s3 = 0.5574


Повторим тоже самое на корпусе твиттов [1]. 

Считываем данные:

In [5]:
import pandas as pd
data = pd.read_csv('/NLP/data/positive.csv', sep=';', header=None,  index_col = False,
                  names = [ 'id', 'tdate', 'tuser', 'ttext', 'ttype',
                          'trep', 'tfav', 'tstcount', 'tfol','tfriend', 'listcount'])
data.head()

,id,tdate,tuser,ttext,ttype,trep,tfav,tstcount,tfol,tfriend,listcount
0,408906692374446080,1386325927,pleease_shut_up,"@first_timee хоть я и школота, но поверь, у на...",1,0,0,0,7569,62,61
1,408906692693221377,1386325927,alinakirpicheva,"Да, все-таки он немного похож на него. Но мой ...",1,0,0,0,11825,59,31
2,408906695083954177,1386325927,EvgeshaRe,RT @KatiaCheh: Ну ты идиотка) я испугалась за ...,1,0,1,0,1273,26,27
3,408906695356973056,1386325927,ikonnikova_21,"RT @digger2912: ""Кто то в углу сидит и погибае...",1,0,1,0,1549,19,17
4,408906761416867842,1386325943,JumpyAlex,@irina_dyshkant Вот что значит страшилка :D\nН...,1,0,0,0,597,16,23


Предобработка (приводим к нижнему регистру, удаляем стоп-слова и некириллические символы):

In [6]:
import re
regex = re.compile("[А-Яа-яё]+")

from nltk.corpus import stopwords

def words_only(text, regex=regex):
    return " ".join(regex.findall(text))



def  remove_stopwords(text, stopwords = stopwords.words('russian')):
    try:
        return " ".join([token for token in text.split() if not token in stopwords])
    except:
        return ""


raw_tweets = data.ttext.tolist()
data.ttext = data.ttext.str.lower()
data.ttext = data.ttext.apply(words_only)
data.ttext = data.ttext.apply(remove_stopwords)   


tweets = [tweet.split() for tweet in data.ttext.tolist()]

Инициализируем класс для вычисления близостей:

In [8]:
from gensim.similarities import WmdSimilarity
num_best = 10
instance = WmdSimilarity(tweets, model, num_best=10)

Задаем документ-запрос и предобрабатываем его:

In [9]:
s1 = 'Ученые обнаружили ископаемую ящерицу с парой теменных глаз'
query = words_only(s1).lower().split()

Поиск по запросу:

In [10]:
sims = instance[query] 

И результаты:

In [11]:
print('Query:', query)
for i in range(10):
    print(sims[i][0])
    print(raw_tweets[sims[i][0]])
    print()

Query: ['ученые', 'обнаружили', 'ископаемую', 'ящерицу', 'с', 'парой', 'теменных', 'глаз']
2401
Ученые установили, что если пищу пережевывать, то давятся витамины, а если не пережевывать - люди.))

12140
@__cherry__bomb_ про девушку у которой врачи обнаружили рак .ну вообще интересный )))

94458
RT @jofutypobaty: Ученые скрестили гиену с пнем и получили гиенологическое дерево)))

29935
Американские ученые выяснили - в недосыпании детей виноваты родители -  http://t.co/NJXMM5DYrA А-а, это так ржачно, надо поделиться )))

109592
@RimVell всеееее, выкололи тебе этот глаз и пролезли в мозг!)

46433
читали? http://t.co/kVaartO6bfПортал:Ниасилили/t.A.T.u )) не в бровь, а в глаз,как говориться) всю статью можно растащить на цитаты)

114425
RT @VITAMIN_PERM: Сначала глаз сломался ... Потом и мозг треснул )))
 http://t.co/V6p6udMSQ4

93210
@Vladiysss мандаринку приложи,говорят помогает)
Сразу глаз пройдёт

56187
@freiman @vodolady отлично,  завтра приду на работу с баклашкой со словами "ученые 

## Источники 
1. Kusner, Matt, Yu Sun, Nicholas Kolkin, and Kilian Weinberger. "From word embeddings to document distances." In International Conference on Machine Learning, pp. 957-966. 2015.
2. Ю. В. Рубцова. Построение корпуса текстов для настройки тонового классификатора // Программные продукты и системы, 2015, №1(109), –С.72-78